# Chap 6. Chaining Prompts

## 1. 提取产品和类别

In [1]:
from tool import get_completion_from_messages

delimiter = "####"

system_message = f"""
您将获得客户服务查询。
客户服务查询将使用{delimiter}字符作为分隔符。
请仅输出一个可解析的Python列表，列表每一个元素是一个JSON对象，每个对象具有以下格式：
'category': <包括以下几个类别：Computers and Laptops、Smartphones and Accessories、Televisions and Home Theater Systems、Gaming Consoles and Accessories、Audio Equipment、Cameras and Camcorders>,
以及
'products': <必须是下面的允许产品列表中找到的产品列表>

类别和产品必须在客户服务查询中找到。
如果提到了某个产品，它必须与允许产品列表中的正确类别关联。
如果未找到任何产品或类别，则输出一个空列表。
除了列表外，不要输出其他任何信息！

允许的产品：

Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Smartphones and Accessories category:
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Gaming Consoles and Accessories category:
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera
    
只输出对象列表，不包含其他内容。。 
"""
user_message_1 = f"""
请告诉我关于 smartx pro phone 和 the fotosnap camera 的信息。
另外，请告诉我关于你们的tvs的情况。
"""
messages = [
  {'role': 'system', 'content': system_message},
  {'role': 'user', 'content': f"{delimiter}{user_message_1}"}
]

category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

[
    {"category": "Smartphones and Accessories", "products": ["SmartX ProPhone"]},
    {"category": "Cameras and Camcorders", "products": ["FotoSnap DSLR Camera", "FotoSnap Mirrorless Camera", "FotoSnap Instant Camera"]},
    {"category": "Televisions and Home Theater Systems", "products": ["CineView 4K TV", "CineView 8K TV", "CineView OLED TV"]}
]


In [2]:
user_message_2 = f"""我的路由器不工作了"""
messages = [
  {'role': 'system', 'content': system_message},
  {'role': 'user', 'content': f"{delimiter}{user_message_2}{delimiter}"}
]
response = get_completion_from_messages(messages)
print(response)

[]


## 2. 检索详细信息

In [3]:
import json
# 读取产品信息
with open("products_zh.json", "r") as file:
  products = json.load(file)

In [4]:
def get_product_by_name(name):
  """
  根据产品名称获取产品

  参数:
  name: 产品名称
  """
  return products.get(name, None)

def get_products_by_category(category):
  """
  根据类别获取产品

  参数:
  category: 产品类别
  """
  return [product for product in products.values() if product['类别'] == category]
  

In [5]:
get_product_by_name("TechPro Ultrabook")

{'名称': 'TechPro 超极本',
 '类别': '电脑和笔记本',
 '品牌': 'TechPro',
 '型号': 'TP-UB100',
 '保修期': '1 year',
 '评分': 4.5,
 '特色': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'],
 '描述': '一款时尚轻便的超极本，适合日常使用。',
 '价格': 799.99}

In [6]:
get_products_by_category("电脑和笔记本")

[{'名称': 'TechPro 超极本',
  '类别': '电脑和笔记本',
  '品牌': 'TechPro',
  '型号': 'TP-UB100',
  '保修期': '1 year',
  '评分': 4.5,
  '特色': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'],
  '描述': '一款时尚轻便的超极本，适合日常使用。',
  '价格': 799.99},
 {'名称': 'BlueWave 游戏本',
  '类别': '电脑和笔记本',
  '品牌': 'BlueWave',
  '型号': 'BW-GL200',
  '保修期': '2 years',
  '评分': 4.7,
  '特色': ['15.6-inch display',
   '16GB RAM',
   '512GB SSD',
   'NVIDIA GeForce RTX 3060'],
  '描述': '一款高性能的游戏笔记本电脑，提供沉浸式体验。',
  '价格': 1199.99},
 {'名称': 'PowerLite Convertible',
  '类别': '电脑和笔记本',
  '品牌': 'PowerLite',
  '型号': 'PL-CV300',
  '保修期': '1 year',
  '评分': 4.3,
  '特色': ['14-inch touchscreen', '8GB RAM', '256GB SSD', '360-degree hinge'],
  '描述': '一款多功能的可转换笔记本电脑，具有灵敏的触摸屏。',
  '价格': 699.99},
 {'名称': 'TechPro Desktop',
  '类别': '电脑和笔记本',
  '品牌': 'TechPro',
  '型号': 'TP-DT500',
  '保修期': '1 year',
  '评分': 4.4,
  '特色': ['Intel Core i7 processor',
   '16GB RAM',
   '1TB HDD',
   'NVIDIA GeForce GTX 1660'],
  '描述': '一款功能强大的台式电脑，适用于工作和娱乐。',
  '价格':

## 3. 生成查询答案

### 3.1. 解析输入字符串 

In [7]:
def read_string_to_list(input_string):
  """
  将输入的字符串转换为 Python 列表。

  参数:
  input_string: 输入的字符串，应为有效的 JSON 格式。

  返回:
  list 或 None: 如果输入字符串有效，则返回对应的 Python 列表，否则返回 None。
  """
  if input_string is None:
    return None
  
  try:
    # 将输入字符串中的单引号替换为双引号，以满足 JSON 格式的要求
    input_string = input_string.replace("'", "\"")
    data = json.loads(input_string)
    return data
  except json.JSONDecodeError:
    print("Error: Invalid JSON string")
    return None
  
category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)
  

[{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera', 'FotoSnap Mirrorless Camera', 'FotoSnap Instant Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'CineView 8K TV', 'CineView OLED TV']}]


### 3.2. 进行检索

In [8]:
def generate_output_string(data_list):
  """
  根据输入的数据列表生成包含产品或类别信息的字符串。

  参数:
  data_list: 包含字典的列表，每个字典都应包含 "products" 或 "category" 的键。

  返回:
  output_string: 包含产品或类别信息的字符串。
  """
  output_string = ""
  if data_list is None:
    return output_string
  
  for data in data_list:
    try:
      if "products" in data and data["products"]:
        product_list = data["products"]
        for product_name in product_list:
          product = get_product_by_name(product_name)
          if product:
            output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
          else:
            print(f"Error: Product '{product_name}' not found")
      elif "category" in data:
        category_name = data["category"]
        category_products = get_products_by_category(category_name)
        for product in category_products:
          output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
      else:
        print("Error: Invalid object format")
    except Exception as e:
      print(f"Error: {e}")
  return output_string

product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

{
    "名称": "SmartX ProPhone",
    "类别": "智能手机和配件",
    "品牌": "SmartX",
    "型号": "SX-PP10",
    "保修期": "1 year",
    "评分": 4.6,
    "特色": [
        "6.1-inch display",
        "128GB storage",
        "12MP dual camera",
        "5G"
    ],
    "描述": "一款拥有先进摄像功能的强大智能手机。",
    "价格": 899.99
}
{
    "名称": "FotoSnap DSLR Camera",
    "类别": "相机和摄像机",
    "品牌": "FotoSnap",
    "型号": "FS-DSLR200",
    "保修期": "1 year",
    "评分": 4.7,
    "特色": [
        "24.2MP sensor",
        "1080p video",
        "3-inch LCD",
        "Interchangeable lenses"
    ],
    "描述": "使用这款多功能的单反相机，捕捉惊艳的照片和视频。",
    "价格": 599.99
}
{
    "名称": "FotoSnap Mirrorless Camera",
    "类别": "相机和摄像机",
    "品牌": "FotoSnap",
    "型号": "FS-ML100",
    "保修期": "1 year",
    "评分": 4.6,
    "特色": [
        "20.1MP sensor",
        "4K video",
        "3-inch touchscreen",
        "Interchangeable lenses"
    ],
    "描述": "一款具有先进功能的小巧轻便的无反相机。",
    "价格": 799.99
}
{
    "名称": "FotoSnap Instant Camera",
    "类别": "相机和摄像机",
    "品牌": 

### 3.3. 生成用户查询的答案

In [9]:
system_message = f"""
您是一家大型电子商店的客服助理。
请以友好和乐于助人的口吻回答问题，并尽量简洁明了。
请确保向用户提出相关的后续问题。
"""

user_message_1 = f"""
请告诉我关于 smartx pro phone 和 the fotosnap camera 的信息。
另外，请告诉我关于你们的tvs的情况。
"""

messages = [
  {'role': 'system', 'content': system_message},
  {'role': 'user', 'content': user_message_1},
  {'role': 'assistant', 'content': f"""相关产品信息：\n\
    {product_information_for_user_message_1}"""}
]

final_repsonse = get_completion_from_messages(messages)
print(final_repsonse)

您好！很高兴为您提供产品信息。以下是您询问的SmartX Pro Phone和Fotosnap相机的详情，以及我们的电视产品概况：

**1. SmartX Pro Phone**  
- **价格**：¥899.99  
- **特点**：6.1英寸屏幕、128GB存储、1200万像素双摄、5G网络  
- **描述**：一款功能强大的智能手机，适合日常使用和高性能需求。  
- **保修**：1年  

**2. Fotosnap相机系列**  
我们提供多款Fotosnap相机，以下是热门型号：  
- **Fotosnap DSLR相机**（¥599.99）：2420万像素、1080p视频、可换镜头  
- **Fotosnap无反相机**（¥799.99）：4K视频、触摸屏、轻便设计  
- **Fotosnap即时相机**（¥69.99）：即拍即印、内置闪光灯，适合趣味拍摄  

**3. 电视产品**  
我们的电视品牌为**CineView**，涵盖多种型号：  
- **4K电视**（¥599.99起）：55英寸、HDR、智能系统  
- **8K电视**（¥2999.99起）：65英寸、超高分辨率  
- **OLED电视**（¥1499.99起）：55英寸、鲜艳色彩、高端画质  

如果您对某一款产品感兴趣（例如具体型号、功能比较或库存情况），请告诉我，我会进一步协助！您是否需要了解某款产品的促销活动或配件推荐？ 😊
